In [ ]:
import re
import urllib.parse as parse

import astropy
import astropy.units as u
import numpy as np
import pandas as pd

In [ ]:
giant_multi_table = pd.read_csv(
    'data/catalogs/GiantMulti_earthunit.csv',
    index_col=['star_id', 'pl_id'],
)

In [ ]:
giant_multi_table

In [ ]:
def format_sig_fig(value, lower, upper):
    lower_abs = abs(lower)
    lower_order = np.floor(np.log10(lower_abs))
    upper_order = np.floor(np.log10(upper))
    value_order = int(np.floor(np.log10(abs(value))))

    unc_order = int(min(lower_order, upper_order)) - 1
    if unc_order <= 1 and value_order >= -3:
        dec_places = abs(unc_order) if unc_order <= 0 else 0
        fmt_str = f'{{:.{dec_places}f}}'
        value_str = fmt_str.format(value)
        lower_str = fmt_str.format(lower_abs)
        upper_str = fmt_str.format(upper)
        if lower_str == upper_str:
            unc_str = f'\\pm {lower_str}'
        else:
            unc_str = f'_{{-{lower_str}}}^{{+{upper_str}}}'
    elif unc_order <= value_order:
        dec_places = abs(value_order - unc_order)
        fmt_str = f'{{:.{dec_places}f}}'
        exponent = 10 ** -value_order
        value_str = '(' + fmt_str.format(value * exponent)
        lower_str = fmt_str.format(lower_abs * exponent)
        upper_str = fmt_str.format(upper * exponent)
        if lower_str == upper_str:
            unc_str = f'\\pm {lower_str}) \\times 10^{{{value_order}}}'
        else:
            unc_str = f'_{{-{lower_str}}}^{{+{upper_str}}}) \\times 10^{{{value_order}}}'
    return value_str, unc_str

def format_sig_fig_single(value):
    if value >= 100:
        return round(value)
    else:
        return round(value, 1)

def texify_citation(citation):
    if citation.startswith('https'):
        m = re.match(r'https://ui.adsabs.harvard.edu/abs/(.+)/abstract', citation)
        bibcode = m.group(1)
        sanitized_bibcode = (parse.unquote(bibcode)).replace('&', 'n')
        return f'\\citet{{{sanitized_bibcode}}}'
    else:
        return citation

In [ ]:
print(texify_citation('https://ui.adsabs.harvard.edu/abs/2022AJ....164...13H/abstract'))

In [ ]:
def texify_catalog(tab):
    lines = []
    for row in tab.itertuples():
        # print(row)
        tex_row = [row.hostname, row.pl_name]
        # flags = []
        for f, l in zip([row.rv_flag, row.tran_flag, row.ttv_flag], 'RTV'):
            if pd.notna(f) and f:
                tex_row.append(l)
            else:
                tex_row.append(' ')
        # tex_row.append('\\textsc{{{}}}'.format(''.join(flags)))
        if pd.notna(row.pl_orbpererr1):
            tex_row.append('${}$'.format(''.join(format_sig_fig(row.pl_orbper, row.pl_orbpererr2, row.pl_orbpererr1))))
        else:
            tex_row.append(f'${row.pl_orbper}$')
        # radius = (np.array([row.pl_rade, row.pl_radeerr2, row.pl_radeerr1])
        if pd.notna(row.pl_rade):
            if pd.notna(row.pl_radeerr1):
                tex_row.append('${}$'.format(''.join(format_sig_fig(row.pl_rade, row.pl_radeerr2, row.pl_radeerr1))))
            else:
                tex_row.append('${}$'.format(format_sig_fig_single(row.pl_rade)))
        else:
            tex_row.append('--')
        if row.pl_masselim == 0:
            # mass = (np.array([row.pl_massj, row.pl_massjerr2, row.pl_massjerr1]) << u.jupiterMass).to(u.earthMass).value
            tex_row.append('${}$'.format(''.join(format_sig_fig(row.pl_masse, row.pl_masseerr2, row.pl_masseerr1))))
        elif row.pl_masselim == 1:
            # tex_row.append('$< {}$'.format(round((row.pl_massj * u.jupiterMass).to(u.earthMass).value)))
            tex_row.append('$< {}$'.format(format_sig_fig_single(row.pl_masse)))
        elif pd.notna(row.pl_msinie):
            tex_row.append('$\\geq {}$'.format(''.join(format_sig_fig(row.pl_msinie, row.pl_msinieerr2, row.pl_msinieerr1))))
        else:
            tex_row.append('--')
        tex_row.append(row.disposition)
        if ';' in row.reference:
            cites = [texify_citation(r) for r in row.reference.split(';')]
            tex_row.append(cites[0])
        else:
            tex_row.append(texify_citation(row.reference))
        lines.append(' & '.join(tex_row) + r' \\')
    return lines

In [ ]:
print('\n'.join(texify_catalog(giant_multi_table)))